In [1]:
import os
import sys
import json

# Ensure local modules can be imported
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import our custom interface classes
from config.config_manager import ConfigManager
from die_studies.auto_die_studies.auto_die_studies_wrapper import AutoDieStudiesWrapper

print("Environment setup complete. Modules loaded.")

Environment setup complete. Modules loaded.


In [2]:
try:
    # Initialize Configuration Manager
    cfg = ConfigManager("config/config.json")

    # Retrieve active settings
    active_pipeline_name = cfg.get_active_pipeline_name()
    target_side = cfg.config.get("analysis_settings", {}).get("target_side", "unknown")

    # Resolve paths based on settings
    raw_source_path = cfg.get_source_image_path()
    work_dir = cfg.get_temp_path()
    output_dir = cfg.get_output_dir()

    # Display summary
    print(f"--- Configuration Summary ---")
    print(f"Target Side:      {target_side.upper()}")
    print(f"Active Pipeline:  {active_pipeline_name}")
    print(f"Source Images:    {raw_source_path}")
    print(f"Working Dir:      {work_dir}")
    print(f"Output Dir:       {output_dir}")
    print(f"-----------------------------")

except Exception as e:
    print(f"Configuration Error: {e}")

--- Configuration Summary ---
Target Side:      OBVERSE
Active Pipeline:  auto_die_studies
Source Images:    D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\bueschelquinare_images_test
Working Dir:      D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\flattened_images
Output Dir:       D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results
-----------------------------


In [3]:
wrapper = None

try:
    # Get specific parameters for the selected pipeline
    pipeline_cfg = cfg.get_pipeline_config(active_pipeline_name)

    print(f"Initializing wrapper for '{active_pipeline_name}'...")

    if active_pipeline_name == "auto_die_studies":
        # Instantiate the Auto-Die-Studies Wrapper
        wrapper = AutoDieStudiesWrapper(
            pipeline_name=active_pipeline_name,
            install_path=pipeline_cfg["install_path"],
            parameters=pipeline_cfg["parameters"],
            raw_source_path=raw_source_path,  # Path resolved by ConfigManager (Obverse/Reverse)
            work_dir=work_dir,
            base_output_dir=output_dir
        )
        print("Wrapper initialized successfully.")

    elif active_pipeline_name == "die_study_tool":
        print("Wrapper for DieStudyTool is not yet implemented in this notebook.")
        # Future implementation:
        # wrapper = DieStudyToolWrapper(...)

    else:
        print(f"Error: Unknown pipeline '{active_pipeline_name}'.")

except Exception as e:
    print(f"Initialization Error: {e}")

Initializing wrapper for 'auto_die_studies'...
Created run output directory: D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results\auto_die_studies\2025-12-09_14-43-50
Wrapper initialized successfully.


In [4]:
if wrapper:
    try:
        print("Starting analysis workflow...")

        # --- RUN THE PIPELINE ---
        # This executes:
        # 1. flatten_image_directory()
        # 2. Pipeline Logic (Matching & Clustering)
        # 3. create_clustering_csv()
        # 4. convert_csv_to_sna_json()
        final_json_path = wrapper.run()

        print(f"\nSUCCESS! Workflow completed.")
        print(f"Final standardized JSON saved to: {final_json_path}")

    except Exception as e:
        print(f"\nCRITICAL ERROR during execution: {e}")
else:
    print("No wrapper initialized. Please check configuration.")

Starting analysis workflow...
--- Pipeline Start: auto_die_studies ---
1. Flattening images from 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\bueschelquinare_images_test' to 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\flattened_images'...
Cleaning content of target directory 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\flattened_images'...
Analyzing source directory 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\bueschelquinare_images_test'...
Found 9 images. Starting copy process...


Flattening: 100%|██████████| 9/9 [00:00<00:00, 642.40img/s]


Process completed.
Successfully copied: 9
Skipped (duplicates): 0
2. Loading Pipeline Modules from D:\Uni\7. Semester\BA\Schnittstelle\Auto-Die-Studies...



D:\Uni\Programmieren\Coin-Die-SNA-Interface\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3. Running Matching (XFeat)...
loading weights from: D:\Uni\7. Semester\BA\Schnittstelle\Auto-Die-Studies\extract_features/../accelerated_features/weights/xfeat.pt


Matching: 100%|██████████| 9/9 [00:20<00:00,  2.31s/it]


Elapsed : 21.691474s
4. Running Clustering...


Computing Silhouettes: 100%|██████████| 103/103 [00:00<00:00, 2727.05it/s]


Optimal threshold : 14
5. Standardizing Results...
Successfully created CSV at 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results\auto_die_studies\2025-12-09_14-43-50\clustering_XFeat_AGLP.csv' with 9 entries.
Successfully converted CSV to JSON: 'D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results\auto_die_studies\2025-12-09_14-43-50\sna_data_XFeat_AGLP.json' (9 entries).
--- Pipeline Finished. Results saved to: D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results\auto_die_studies\2025-12-09_14-43-50 ---

SUCCESS! Workflow completed.
Final standardized JSON saved to: D:\Uni\7. Semester\BA\Schnittstelle\Coin-Die-SNA-Interface\results\auto_die_studies\2025-12-09_14-43-50\sna_data_XFeat_AGLP.json
